# Bologna TSP

This uses the code from [these slides from Bologna Business School](http://cs.unibo.it/~tong.liu3/mzn/slides_mzn.pdf).

In [1]:
 %load_ext iminizinc

<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.2.1, build 29527117
Copyright (C) 2014-2018 Monash University, NICTA, Data61


In [4]:
import pandas as pd
from geopy.distance import great_circle
from IPython.display import display

In [5]:
cities = pd.read_csv("data/city.csv")

In [7]:
type(cities)

pandas.core.frame.DataFrame

In [9]:
cdict = cities.set_index('City').T.to_dict('list')

In [29]:
keys = list(cdict.keys())
# Can't do all 48 in a reasonable amount of time! 
keys = keys[:14] # 14 is the limit for < 1 minute run time.

# Here are values for the variables/parameters shared with MiniZinc:
n = len(keys)
dist = [[int(great_circle(tuple(cdict[key1]), tuple(cdict[key2])).miles)
           for key1 in keys]
          for key2 in keys]
city_name = keys
start_city = keys.index('Hartford') + 1 #
end_city = keys.index('Dover') + 1
print('n = ',n, '\ncity_name =', city_name, '\nstart_city = ', start_city, '\nend_city = ',end_city)

n =  14 
city_name = ['Montgomery', 'Phoenix', 'Little Rock', 'Sacramento', 'Denver', 'Hartford', 'Dover', 'Tallahassee', 'Atlanta', 'Boise', 'Springfield', 'Indianapolis', 'Des Moines', 'Topeka'] 
start_city =  6 
end_city =  7


In [30]:
%%minizinc -m bind
include "globals.mzn";

int: n;
array [1..n,1..n] of int: dist;
int: start_city;
int: end_city;

array[1..n] of var 1..n: city;
    
constraint city[1] = start_city;
constraint city[n] = end_city;
constraint all_different(city);

var int: total_distance = sum(i in 2..n)(dist[city[i-1],city[i]]);
solve minimize total_distance;

In [31]:
city

[6, 12, 11, 13, 14, 5, 10, 4, 2, 3, 1, 8, 9, 7]

In [32]:
[city_name[i-1] for i in city]

['Hartford',
 'Indianapolis',
 'Springfield',
 'Des Moines',
 'Topeka',
 'Denver',
 'Boise',
 'Sacramento',
 'Phoenix',
 'Little Rock',
 'Montgomery',
 'Tallahassee',
 'Atlanta',
 'Dover']

## Bibliography

* [GitHub site for TSP solved for 48 US States.](https://github.com/toddwschneider/shiny-salesman) But the data is in binary. 
* [Here is longitudes and latitudes](http://www.xfront.com/us_states/), then use [this](https://pypi.org/project/geopy/) to calculate the distances. 